In [1]:
ls

 Volume in drive C is Windows
 Volume Serial Number is E2EC-E904

 Directory of c:\Users\T\Desktop\Al_Project\Traffic-Sign-Detection-Using-CNN

03/06/2025  09:37 CH    <DIR>          .
26/05/2025  07:35 SA    <DIR>          ..
03/05/2025  03:24 CH                12 .gitignore
03/06/2025  09:38 CH         3.773.868 best_model.h5
03/06/2025  09:22 CH    <DIR>          dataset1
03/06/2025  09:25 CH    <DIR>          dataset2
03/06/2025  09:37 CH    <DIR>          dataset3
03/05/2025  08:32 SA             1.096 LICENSE
03/06/2025  09:37 CH             4.490 main.py
25/05/2025  09:15 SA         3.773.852 model.h5
03/05/2025  09:09 SA        18.785.573 model.keras
03/05/2025  03:33 CH               176 README.md
03/05/2025  08:58 SA                98 requirements.txt
18/05/2025  10:25 CH             4.754 T�m t?t code.py
03/06/2025  09:31 CH           146.964 Training1.ipynb
03/06/2025  09:28 CH                 0 Training2.ipynb
03/05/2025  08:41 SA    <DIR>          venv
              11 Fi

In [2]:
import os
import shutil
import random

# ================== Cấu hình ================== #
DATA_DIR = 'dataset3/traffic_data/data'
OUTPUT_DIR = 'dataset3'
TRAIN_RATIO = 0.7
VAL_RATIO = 0.15
TEST_RATIO = 0.15
SEED = 42

# ================== Tạo thư mục ================== #
def create_dir(path):
    if not os.path.exists(path):
        os.makedirs(path)

for folder in ['train', 'val', 'test']:
    for class_name in os.listdir(DATA_DIR):
        create_dir(os.path.join(OUTPUT_DIR, folder, class_name))

# ================== Chia dữ liệu ================== #
random.seed(SEED)

for class_name in os.listdir(DATA_DIR):
    class_path = os.path.join(DATA_DIR, class_name)
    if not os.path.isdir(class_path):
        continue

    images = os.listdir(class_path)
    random.shuffle(images)

    total = len(images)
    train_end = int(total * TRAIN_RATIO)
    val_end = train_end + int(total * VAL_RATIO)

    train_imgs = images[:train_end]
    val_imgs = images[train_end:val_end]
    test_imgs = images[val_end:]

    # Copy ảnh vào thư mục tương ứng
    for img_name in train_imgs:
        shutil.copy2(os.path.join(class_path, img_name),
                     os.path.join(OUTPUT_DIR, 'train', class_name, img_name))

    for img_name in val_imgs:
        shutil.copy2(os.path.join(class_path, img_name),
                     os.path.join(OUTPUT_DIR, 'val', class_name, img_name))

    for img_name in test_imgs:
        shutil.copy2(os.path.join(class_path, img_name),
                     os.path.join(OUTPUT_DIR, 'test', class_name, img_name))

print("✅ Đã chia xong dữ liệu thành train/val/test.")


✅ Đã chia xong dữ liệu thành train/val/test.


In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D, MaxPool2D, Dropout, BatchNormalization, GlobalAveragePooling2D, Dense
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.callbacks import EarlyStopping, ReduceLROnPlateau, ModelCheckpoint
from tensorflow.keras.optimizers import Adam
import tensorflow.keras.backend as K
from sklearn.metrics import confusion_matrix, classification_report

# ===================== Cấu hình dữ liệu ===================== #
image_size = (30, 30)
batch_size = 32
num_classes = 58

train_dir = 'dataset2/train'
val_dir = 'dataset2/val'
test_dir = 'dataset2/test'

# ===================== Tạo ImageDataGenerator ===================== #
train_datagen = ImageDataGenerator(
    rescale=1./255,
    rotation_range=15,
    zoom_range=0.1,
    width_shift_range=0.1,
    height_shift_range=0.1,
    shear_range=0.1,
    horizontal_flip=False,
    fill_mode='nearest'
)

val_test_datagen = ImageDataGenerator(rescale=1./255)

train_generator = train_datagen.flow_from_directory(
    train_dir,
    target_size=image_size,
    batch_size=batch_size,
    class_mode='categorical',
    shuffle=True  # Nên bật shuffle cho train
)

val_generator = val_test_datagen.flow_from_directory(
    val_dir,
    target_size=image_size,
    batch_size=batch_size,
    class_mode='categorical',
    shuffle=False  # Tắt shuffle cho validation để có thể so sánh chính xác
)

test_generator = val_test_datagen.flow_from_directory(
    test_dir,
    target_size=image_size,
    batch_size=batch_size,
    class_mode='categorical',
    shuffle=False  # Rất quan trọng giữ đúng thứ tự nhãn
)

# ===================== Định nghĩa custom metric RMSE =====================
def rmse(y_true, y_pred):
    return K.sqrt(K.mean(K.square(y_pred - y_true)))

# ===================== Xây dựng model =====================
model = Sequential([
    Conv2D(64, (3, 3), padding='same', activation='relu', input_shape=(image_size[0], image_size[1], 3)),
    BatchNormalization(),
    Dropout(0.1),

    Conv2D(64, (3, 3), padding='same', activation='relu'),
    BatchNormalization(),
    MaxPool2D((2, 2)),
    Dropout(0.25),

    Conv2D(128, (3, 3), padding='same', activation='relu'),
    BatchNormalization(),
    Dropout(0.1),

    Conv2D(128, (3, 3), padding='same', activation='relu'),
    BatchNormalization(),
    MaxPool2D((2, 2)),
    Dropout(0.25),

    GlobalAveragePooling2D(),

    Dense(256, activation='relu'),
    BatchNormalization(),
    Dropout(0.5),

    Dense(num_classes, activation='softmax')
])

model.summary()

# ===================== Compile model =====================
model.compile(
    optimizer=Adam(),  # Dùng Adam mặc định
    loss='categorical_crossentropy',
    metrics=['accuracy', 'mae', 'mse', 'mape', rmse]
)

# ===================== Callbacks =====================
early_stopping = EarlyStopping(monitor='val_loss', patience=7, restore_best_weights=True)
reduce_lr = ReduceLROnPlateau(monitor='val_loss', factor=0.5, patience=3, verbose=1)
model_checkpoint = ModelCheckpoint('best_model2.h5', monitor='val_loss', save_best_only=True, verbose=1)

# ===================== Train model =====================
epochs = 100

history = model.fit(
    train_generator,
    steps_per_epoch=train_generator.samples // batch_size,
    validation_data=val_generator,
    validation_steps=val_generator.samples // batch_size,
    epochs=epochs,
    callbacks=[early_stopping, reduce_lr, model_checkpoint]
)

# ===================== Đánh giá model và tạo ma trận nhầm lẫn =====================

# Lưu mô hình cuối cùng
model.save('model2.h5')

# Dự đoán trên tập test
test_steps = (test_generator.samples + batch_size - 1) // batch_size  # làm tròn lên
y_pred_prob = model.predict(test_generator, steps=test_steps)
y_pred = np.argmax(y_pred_prob, axis=1)

# Lấy nhãn thật
y_true = test_generator.classes

# Tính ma trận nhầm lẫn
cm = confusion_matrix(y_true, y_pred)

# Vẽ ma trận nhầm lẫn
plt.figure(figsize=(15, 15))
sns.heatmap(cm, annot=True, fmt='d', cmap='Blues')
plt.xlabel('Dự đoán')
plt.ylabel('Thực tế')
plt.title('Ma trận nhầm lẫn trên tập test')
plt.show()

# In báo cáo phân loại
target_names = list(test_generator.class_indices.keys())
print(classification_report(y_true, y_pred, target_names=target_names))


Found 4176 images belonging to 58 classes.
Found 872 images belonging to 58 classes.
Found 950 images belonging to 58 classes.


Model: "sequential_3"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ conv2d_12 (Conv2D)              │ (None, 30, 30, 64)     │         1,792 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ batch_normalization_15          │ (None, 30, 30, 64)     │           256 │
│ (BatchNormalization)            │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout_15 (Dropout)            │ (None, 30, 30, 64)     │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d_13 (Conv2D)              │ (None, 30, 30, 64)     │        36,928 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ batch_normalization_16          │ (None, 30, 30, 64)     │           256 │
│ (BatchNormalization)            │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling2d_6 (MaxPooling2D)  │ (None, 15, 15, 64)     │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout_16 (Dropout)            │ (None, 15, 15, 64)     │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d_14 (Conv2D)              │ (None, 15, 15, 128)    │        73,856 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ batch_normalization_17          │ (None, 15, 15, 128)    │           512 │
│ (BatchNormalization)            │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout_17 (Dropout)            │ (None, 15, 15, 128)    │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d_15 (Conv2D)              │ (None, 15, 15, 128)    │       147,584 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ batch_normalization_18          │ (None, 15, 15, 128)    │           512 │
│ (BatchNormalization)            │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling2d_7 (MaxPooling2D)  │ (None, 7, 7, 128)      │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout_18 (Dropout)            │ (None, 7, 7, 128)      │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ global_average_pooling2d_3      │ (None, 128)            │             0 │
│ (GlobalAveragePooling2D)        │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_6 (Dense)                 │ (None, 256)            │        33,024 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ batch_normalization_19          │ (None, 256)            │         1,024 │
│ (BatchNormalization)            │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout_19 (Dropout)            │ (None, 256)            │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_7 (Dense)                 │ (None, 58)             │        14,906 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 310,650 (1.19 MB)

 Trainable params: 309,370 (1.18 MB)

...

130/130 ━━━━━━━━━━━━━━━━━━━━ 39s 297ms/step - accuracy: 0.9167 - loss: 0.2762 - mae: 0.0053 - mape: 2628311.7500 - mse: 0.0021 - rmse: 0.0442 - val_accuracy: 0.9479 - val_loss: 0.1709 - val_mae: 0.0029 - val_mape: 1468253.5000 - val_mse: 0.0013 - val_rmse: 0.0258 - learning_rate: 6.2500e-05
Epoch 50/100
  1/130 ━━━━━━━━━━━━━━━━━━━━ 16s 130ms/step - accuracy: 0.9688 - loss: 0.2172 - mae: 0.0053 - mape: 2668679.0000 - mse: 0.0017 - rmse: 0.0411
Epoch 50: val_loss did not improve from 0.17086
130/130 ━━━━━━━━━━━━━━━━━━━━ 4s 31ms/step - accuracy: 0.9688 - loss: 0.2172 - mae: 0.0053 - mape: 2668679.0000 - mse: 0.0017 - rmse: 0.0411 - val_accuracy: 0.9479 - val_loss: 0.1710 - val_mae: 0.0029 - val_mape: 1464973.7500 - val_mse: 0.0013 - val_rmse: 0.0258 - learning_rate: 6.2500e-05
Epoch 51/100
130/130 ━━━━━━━━━━━━━━━━━━━━ 0s 259ms/step - accuracy: 0.9243 - loss: 0.2562 - mae: 0.0051 - mape: 2535089.2500 - mse: 0.0019 - rmse: 0.0432
Epoch 51: val_loss improved from 0.17086 to 0.16650, saving m

130/130 ━━━━━━━━━━━━━━━━━━━━ 37s 283ms/step - accuracy: 0.9244 - loss: 0.2562 - mae: 0.0051 - mape: 2534403.2500 - mse: 0.0019 - rmse: 0.0432 - val_accuracy: 0.9468 - val_loss: 0.1665 - val_mae: 0.0029 - val_mape: 1451696.0000 - val_mse: 0.0012 - val_rmse: 0.0256 - learning_rate: 6.2500e-05
Epoch 52/100
  1/130 ━━━━━━━━━━━━━━━━━━━━ 24s 191ms/step - accuracy: 0.9062 - loss: 0.3480 - mae: 0.0071 - mape: 3553571.7500 - mse: 0.0028 - rmse: 0.0530
Epoch 52: val_loss did not improve from 0.16650
130/130 ━━━━━━━━━━━━━━━━━━━━ 3s 22ms/step - accuracy: 0.9062 - loss: 0.3480 - mae: 0.0071 - mape: 3553571.7500 - mse: 0.0028 - rmse: 0.0530 - val_accuracy: 0.9468 - val_loss: 0.1668 - val_mae: 0.0029 - val_mape: 1452027.3750 - val_mse: 0.0012 - val_rmse: 0.0255 - learning_rate: 6.2500e-05
Epoch 53/100
130/130 ━━━━━━━━━━━━━━━━━━━━ 0s 279ms/step - accuracy: 0.9337 - loss: 0.2474 - mae: 0.0049 - mape: 2461010.0000 - mse: 0.0018 - rmse: 0.0417
Epoch 53: val_loss did not improve from 0.16650
130/130 ━━━━━

130/130 ━━━━━━━━━━━━━━━━━━━━ 42s 318ms/step - accuracy: 0.9434 - loss: 0.2254 - mae: 0.0045 - mape: 2264593.5000 - mse: 0.0017 - rmse: 0.0398 - val_accuracy: 0.9525 - val_loss: 0.1550 - val_mae: 0.0027 - val_mape: 1329207.2500 - val_mse: 0.0012 - val_rmse: 0.0244 - learning_rate: 3.1250e-05
Epoch 56/100
  1/130 ━━━━━━━━━━━━━━━━━━━━ 17s 134ms/step - accuracy: 0.9375 - loss: 0.1862 - mae: 0.0039 - mape: 1961031.3750 - mse: 0.0013 - rmse: 0.0367
Epoch 56: val_loss improved from 0.15500 to 0.15465, saving model to best_model.h5


130/130 ━━━━━━━━━━━━━━━━━━━━ 5s 34ms/step - accuracy: 0.9375 - loss: 0.1862 - mae: 0.0039 - mape: 1961031.3750 - mse: 0.0013 - rmse: 0.0367 - val_accuracy: 0.9525 - val_loss: 0.1546 - val_mae: 0.0027 - val_mape: 1326925.8750 - val_mse: 0.0012 - val_rmse: 0.0243 - learning_rate: 3.1250e-05
Epoch 57/100
130/130 ━━━━━━━━━━━━━━━━━━━━ 0s 264ms/step - accuracy: 0.9399 - loss: 0.2173 - mae: 0.0046 - mape: 2280200.2500 - mse: 0.0016 - rmse: 0.0392
Epoch 57: val_loss improved from 0.15465 to 0.15380, saving model to best_model.h5


OSError: [Errno 0] Unable to synchronously create file (unable to lock file, errno = 0, error message = 'No error', Win32 GetLastError() = 33)